ANÁLISIS NUMÉRICO
TAREA 2
Diego López Zanella
$$ x = 1$$

# PROBLEMA 1



Dados los siguientes vectores:
$$\alpha = \begin{pmatrix} 4 \\ 3  \end{pmatrix}, \;\; \beta = \begin{pmatrix} 2 \\ 1 \\ 2  \end{pmatrix}$$

encuentre la matriz de rotación $G$ y el reflector de Householder $F$ tales que 

$$G \alpha = \begin{pmatrix} 5 \\ 0  \end{pmatrix}, \;\; F \beta = \begin{pmatrix} -3 \\ 0 \\ 0  \end{pmatrix}$$